In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [85]:
from zipfile import ZipFile
with ZipFile('Data/test.zip', 'r') as zipObj:
  zipObj.extractall('Data')

In [37]:
import sys
sys.path.append('/content/drive/MyDrive/MIE479/Deliverable_4')
!ls '/content/drive/MyDrive/MIE479/Deliverable_4'

 linear_svm_2020.pkl  'Linear SVM Model.ipynb'	 rbf_svm_2021.pkl
 linear_svm_2021.pkl   poly2_svm_2021.pkl	'Thunder SVM Models.ipynb'


In [38]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [86]:
file_name = '/content/Data/test.csv'
table = pd.read_csv(file_name, low_memory=False).dropna()

table = table[[
    'ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B', # 'CSCORE_C',
    'FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase', 'NUM_UNIT',
    'OCC_Principal', 'OCC_Second', 'OCC_Investor', 'MI_PCT', 'DLQ_30_FLAG', 'DLQ_90_FLAG',
    'Ongoing', 'Current_DLQ', 'Prepaid_Matured', 'Deed_In_Lieu']]

table['BAD'] = None
table.loc[(table['DLQ_90_FLAG'] == 1) | (table['Deed_In_Lieu'] == 1), 'BAD'] = 1
table.loc[(table['Ongoing'] == 1) & (table['DLQ_30_FLAG'] == 0), 'BAD'] = 0
table = table.dropna(subset=['BAD'])
table['BAD'] = table['BAD'].astype(int)

<ipython-input-86-29d1ac4fa559>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['BAD'] = None
<ipython-input-86-29d1ac4fa559>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['BAD'] = table['BAD'].astype(int)


In [87]:
n_row, n_col = table.shape
n_dlq_90 = sum(table['DLQ_90_FLAG'])
n_fc = sum(table['Deed_In_Lieu'])
print(f"Number of rows: {n_row}")
print(f"Number of columns: {n_col}")
print(f"Percentage of 90-day delinquency: {round(n_dlq_90 / n_row * 100, 2)} %")
print(f"Percentage of foreclosure: {round(n_fc / n_row * 100, 2)} %")
print("The total number of NA is ", sum(table.isna().sum()))

# 2022: 1,751,215. 1.55 %
# 2021: 3,896,571. 1.45 %
# 2020: 3,041,599. 3.07 %
# 2019: 654,073. 18.26 %

# 2020 1st: 964,942. 6.19 %
# 2020 2nd: 2,076,657. 1.61 %

Number of rows: 2165331
Number of columns: 23
Percentage of 90-day delinquency: 5.25 %
Percentage of foreclosure: 0.02 %
The total number of NA is  0


In [88]:
X = table.drop(columns=['DLQ_30_FLAG', 'DLQ_90_FLAG', 'Ongoing', 'Current_DLQ',
                        'Prepaid_Matured', 'Deed_In_Lieu', 'BAD'])
y = table['BAD']

num_col = ['ORIG_RATE', 'ORIG_AMOUNT', 'ORIG_TERM', 'OLTV', 'NUM_BO', 'DTI', 'CSCORE_B', # 'CSCORE_C',
           'NUM_UNIT', 'MI_PCT']
cat_col = ['FTHB_FLAG', 'PUR_Cash_out', 'PUR_Refinance', 'PUR_Purchase',
           'OCC_Principal', 'OCC_Second', 'OCC_Investor']

scaler = StandardScaler()
X[num_col] = scaler.fit_transform(X[num_col])
X[cat_col] = scaler.fit_transform(X[cat_col])
X = pd.concat([X[num_col], X[cat_col]], axis=1)

X

,ORIG_RATE,ORIG_AMOUNT,ORIG_TERM,OLTV,NUM_BO,DTI,CSCORE_B,NUM_UNIT,MI_PCT,FTHB_FLAG,PUR_Cash_out,PUR_Refinance,PUR_Purchase,OCC_Principal,OCC_Second,OCC_Investor
0,0.580453,1.035122,0.586268,0.786336,0.968845,-0.731499,0.274714,-0.108845,0.522873,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
2,1.313286,0.550895,0.586268,0.729562,0.968845,1.698761,-0.610603,-0.108845,0.522873,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
3,1.795412,-0.847114,0.586268,0.672789,-0.966096,-0.731499,-1.594289,-0.108845,0.522873,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
4,0.349032,0.410313,0.586268,1.070203,-0.966096,-0.528978,0.520635,-0.108845,1.675314,2.295197,-0.492185,-0.936091,1.399595,0.309058,-0.194649,-0.231062
15,1.544706,0.058859,0.586268,1.070203,-0.966096,1.597500,-0.069576,-0.108845,1.675314,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993797,-0.615221,0.652427,0.586268,-1.825245,-0.966096,-0.528978,0.963293,-0.108845,-0.540919,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
3993798,-1.097348,-0.175445,0.586268,-1.314284,0.968845,-1.845369,0.987886,-0.108845,-0.540919,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
3993799,0.117612,0.636806,0.586268,-1.314284,0.968845,0.179848,0.077977,-0.108845,-0.540919,-0.435692,-0.492185,1.068272,-0.714492,0.309058,-0.194649,-0.231062
3993800,-1.348053,-0.839304,0.586268,-1.200737,-0.966096,0.584891,1.110846,-0.108845,-0.540919,2.295197,-0.492185,-0.936091,1.399595,0.309058,-0.194649,-0.231062


In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

linear_svm = LinearSVC(random_state=999, max_iter=1000000, C=0.1, class_weight='balanced')
linear_svm.fit(X_train, y_train)

y_pred = linear_svm.predict(X_test)
y_pred_prob = linear_svm.decision_function(X_test)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
roc_auc_prob = roc_auc_score(y_test, y_pred_prob)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"AUC-ROC Predict: {round(roc_auc, 4)}")
print(f"AUC-ROC Decision function: {round(roc_auc_prob, 4)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)

#### 2022:
# Accuracy: 0.7111
# AUC-ROC Predict: 0.7388
# AUC-ROC Decision function: 0.8103
# Classification Report:
#               precision    recall  f1-score   support
#
#            0       0.99      0.71      0.83    517249
#            1       0.04      0.77      0.08      8116
#
#     accuracy                           0.71    525365
#    macro avg       0.52      0.74      0.45    525365
# weighted avg       0.98      0.71      0.82    525365


#### 2021:
# Accuracy: 0.7311
# AUC-ROC Predict: 0.7516
# AUC-ROC Decision function: 0.8233
# Classification Report:
#               precision    recall  f1-score   support
#
#            0       1.00      0.73      0.84   1152123
#            1       0.04      0.77      0.08     16849
#
#     accuracy                           0.73   1168972
#    macro avg       0.52      0.75      0.46   1168972
# weighted avg       0.98      0.73      0.83   1168972


#### 2020:
# Accuracy: 0.7692
# AUC-ROC Predict: 0.7682
# AUC-ROC Decision function: 0.8456
# Classification Report:
#               precision    recall  f1-score   support
#
#            0       0.99      0.77      0.87    884368
#            1       0.10      0.77      0.17     28112
#
#     accuracy                           0.77    912480
#    macro avg       0.54      0.77      0.52    912480
# weighted avg       0.96      0.77      0.84    912480


#### 2019:
# Accuracy: 0.7252
# AUC-ROC Predict: 0.7317
# AUC-ROC Decision function: 0.8054
# Classification Report:
#               precision    recall  f1-score   support
#
#            0       0.93      0.72      0.81    160193
#            1       0.37      0.74      0.50     36029
#
#     accuracy                           0.73    196222
#    macro avg       0.65      0.73      0.65    196222
# weighted avg       0.82      0.73      0.75    196222

# 2020 1st: 0.7431, 0.7439
# 2020 2nd: 0.7466, 0.7546

Accuracy: 0.7654
AUC-ROC Predict: 0.7631
AUC-ROC Decision function: 0.8421
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.77      0.86    615247
           1       0.15      0.76      0.26     34353

    accuracy                           0.77    649600
   macro avg       0.57      0.76      0.56    649600
weighted avg       0.94      0.77      0.83    649600



In [65]:
print(round(y_test.sum() / len(y_test), 4))
print(round(y_pred.sum() / len(y_pred), 4))

import pickle
with open('/content/drive/MyDrive/MIE479/Deliverable_4/linear_svm_2022.pkl', 'wb') as file:
    pickle.dump(linear_svm, file)

# 2022: 0.0154 / 0.2972
# 2021: 0.0144 / 0.2767
# 2020: 0.0308 / 0.2473
# 2019: 0.1836 / 0.3636

0.0154
0.2972


In [90]:
coefficients = linear_svm.coef_
intercept = linear_svm.intercept_
print("Coefficients (w):")
print(coefficients)
print("Intercept (b):")
print(intercept)

#### 2022:
# [[ 0.00746019  0.05517524  0.06425339  0.10526682 -0.12928178  0.11941902
#   -0.38943409 -0.00874112  0.06903505 -0.01499787  0.04282975  0.00691703
#   -0.04369484  0.00804527 -0.01079705 -0.00254444]]
# [-0.29403858]

#### 2021:
# [[ 0.05814352  0.03529345  0.03839151  0.08992641 -0.12984053  0.12068866
#   -0.32720546  0.01279378  0.06806234  0.00111534  0.01413292 -0.0085435
#   -0.00396214  0.01522191 -0.00551465 -0.01465152]]
# [-0.30959152]

#### 2020:
# [[ 0.29618500  0.11011156 -0.04719625  0.06297977 -0.11510221  0.14822568
#   -0.22551190  0.00408573  0.06296063 -0.00110927 -0.00615793  0.00216592
#    0.00286307  0.02918874 -0.00014069 -0.03706380]]
# [-0.34355674]

#### 2019:
# [[ 0.19574152  0.22004727  0.02007519  0.08159825 -0.07992706  0.16068632
#   -0.24288787 -0.00360812  0.05788112 -0.00851677  0.02623715  0.01943688
#   -0.03825022  0.012313   -0.0111734  -0.00633767]]
# [-0.18393624]

#### 2020 1st:
# [[ 0.27704804  0.16137004 -0.04053907  0.05491134 -0.0996026   0.16533096
#   -0.20631185 -0.00212192  0.05162216 -0.00549512  0.0014176   0.00685618
#   -0.00861885  0.02212737  0.00282158 -0.03027004]]
# [-0.27102256]

#### 2020 2nd:
# [[ 0.09283456  0.04813065  0.02314234  0.09322656 -0.13869488  0.14288881
#   -0.28751672  0.01914375  0.06832035  0.00644264  0.00259182 -0.00519779
#    0.00348569  0.01947555 -0.01340041 -0.0135874 ]]
# [-0.32092784]

Coefficients (w):
[[ 0.34283637  0.14755367 -0.05374182  0.05684508 -0.10128123  0.15984986
  -0.20440409 -0.00214302  0.0557899  -0.0024362  -0.00234073  0.01154311
  -0.01021439  0.03009741  0.00072301 -0.03932706]]
Intercept (b):
[-0.32410868]
